In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = 'drive/MyDrive/ML-Oxford/StructureML'

In [ ]:
# Import data

X_data = np.load(path+'/structures_X_test.npy')
Y_data = np.load(path+'/structures_Y_test.npy')

labels = ['benzene', 'cyclohexane', 'cyclopentane', 'tetrahydrofuran',
          'cyclopentadiene', 'furan', 'pyrrole', 'pyrrolidine', 'piperidine',
          'cyclohexanone', 'pyridine']

print(X_data.shape, Y_data.shape)

(1296, 30, 30) (1296,)


In [ ]:
# reshape data to 1 dimension

X_reshaped = X_data.reshape(len(X_data), 900)
X_reshaped = X_reshaped.astype('float32')

print(X_reshaped.shape)

(1296, 900)


In [ ]:
# one-hot encode labels
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
Y_encoded = label_encoder.fit_transform(Y_data)


In [ ]:
from scipy.sparse import random
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X_reshaped, Y_encoded, test_size=0.2, random_state=42, shuffle=True)

print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(1036, 900) (260, 900) (1036,) (260,)


In [ ]:
# Normalise inputs between 0 and 1

X_train /= 255
X_test /= 255

print(X_train[0])

[0.00392157 0.00392157 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.00392157 0.00392157
 0.00392157 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0. 

In [ ]:
# One-hot encode labels

import tensorflow as tf
import numpy as np
from tensorflow import keras

NB_CLASSES = 11

Y_train = tf.keras.utils.to_categorical(Y_train, NB_CLASSES)
Y_test = tf.keras.utils.to_categorical(Y_test, NB_CLASSES)

print(Y_train[0])

[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]


In [ ]:
# Build a baseline model

baseline_model = tf.keras.models.Sequential()
baseline_model.add(keras.layers.Dense(NB_CLASSES,
                             input_shape=(900,),
                             name='dense_layer',
                             activation='softmax'))

In [ ]:
# Compile the baseline model

baseline_model.compile(optimizer='SGD',
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])

In [ ]:
# Train the baseline model

baseline_model.fit(X_train, Y_train, batch_size=32,
                   epochs=50, verbose=True, validation_split=0.2)

Epoch 1/50
26/26 [==============================] - 1s 19ms/step - loss: 2.3601 - accuracy: 0.1425 - val_loss: 2.3631 - val_accuracy: 0.1490
Epoch 2/50
26/26 [==============================] - 0s 3ms/step - loss: 2.3119 - accuracy: 0.1715 - val_loss: 2.3340 - val_accuracy: 0.1635
Epoch 3/50
26/26 [==============================] - 0s 4ms/step - loss: 2.2788 - accuracy: 0.1824 - val_loss: 2.3140 - val_accuracy: 0.1779
Epoch 4/50
26/26 [==============================] - 0s 3ms/step - loss: 2.2535 - accuracy: 0.1836 - val_loss: 2.2975 - val_accuracy: 0.1875
Epoch 5/50
26/26 [==============================] - 0s 3ms/step - loss: 2.2307 - accuracy: 0.1920 - val_loss: 2.2827 - val_accuracy: 0.1923
Epoch 6/50
26/26 [==============================] - 0s 3ms/step - loss: 2.2102 - accuracy: 0.2005 - val_loss: 2.2685 - val_accuracy: 0.1923
Epoch 7/50
26/26 [==============================] - 0s 3ms/step - loss: 2.1906 - accuracy: 0.2077 - val_loss: 2.2551 - val_accuracy: 0.2019
Epoch 8/50
26/26 [=

In [ ]:
test_loss, test_acc = baseline_model.evaluate(X_test, Y_test)
print(f'baseline model test accuracy: {round(test_acc, 3)*100}%')

9/9 [==============================] - 0s 2ms/step - loss: 1.9192 - accuracy: 0.3115
baseline model test accuracy: 31.2%


The dataset is compatible with tf and keras and can be used for more in-depth model engineering

In [ ]:
# Export datasets

np.save(path+'/X_train.npy', X_train)
np.save(path+'/X_test.npy', X_test)
np.save(path+'/Y_train.npy', Y_train)
np.save(path+'/Y_test.npy', Y_test)